# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
from pprint import pprint


# set figure size to 9x5
plt.rcParams['figure.figsize'] = [9, 5]


# Import API key
from api_keys import g_key
# g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
weather_datafile = "../WeatherPy/output_data/weather_by_city.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_datafile)

# Combine the data into a single dataset

weather_df = pd.DataFrame(weather_data)
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed
0,Cape Town,90,ZA,1586658870,93,-33.93,18.42,64.99,27.51
1,Albany,1,US,1586658894,50,42.60,-73.97,39.99,6.93
2,Bluff,20,NZ,1586658894,51,-46.60,168.33,71.01,5.99
3,Verkhniy Baskunchak,0,RU,1586658894,73,48.23,46.72,38.26,16.71
4,Inírida,64,CO,1586658894,88,3.87,-67.92,75.88,1.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
### Use the Lat and Lng as locations and Humidity as the weight.
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Latitude", "Longitude"]].astype(float)
locations.head() 




,Latitude,Longitude
0,-33.93,18.42
1,42.60,-73.97
2,-46.60,168.33
3,48.23,46.72
4,3.87,-67.92


In [5]:
hum_rate = weather_df["Humidity"].astype(float)
hum_rate.head()

0    93.0
1    50.0
2    51.0
3    73.0
4    88.0
Name: Humidity, dtype: float64

In [6]:

# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("Humidity_HeatMap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 648x360 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
### Narrow down the DataFrame to find your ideal weather condition.
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph.
# Zero cloudiness.

#MY ideal weather : A max temperature lower than 80 degrees but higher than 70
ideal_weather_df = weather_df[(weather_df['Temperature'] >= 70)
                  & (weather_df['Temperature'] < 80) 
                  & (weather_df['Wind Speed'] < 10) 
                  & (weather_df['Cloudiness'] == 0)]


ideal_weather_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed
9,Barra de Tecoanapa,0,MX,1586658896,82,16.52,-98.75,77.86,4.29
57,Port Macquarie,0,AU,1586658909,37,-31.43,152.92,73.00,6.93
82,Huarmey,0,PE,1586658748,72,-10.07,-78.15,75.13,7.14
425,Ambilobe,0,MG,1586659017,82,-13.20,49.05,71.83,4.74
430,Tessalit,0,ML,1586659018,14,20.20,1.01,77.38,4.32
471,Ballitoville,0,ZA,1586659029,93,-29.54,31.21,72.00,2.24
495,Mpophomeni,0,ZA,1586659036,66,-29.57,30.19,72.00,4.97


In [8]:
ideal_weather_df.count()

City           7
Cloudiness     7
Country        7
Date           7
Humidity       7
Latitude       7
Longitude      7
Temperature    7
Wind Speed     7
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

hotel_df = ideal_weather_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed,Hotel Name
9,Barra de Tecoanapa,0,MX,1586658896,82,16.52,-98.75,77.86,4.29,
57,Port Macquarie,0,AU,1586658909,37,-31.43,152.92,73.00,6.93,
82,Huarmey,0,PE,1586658748,72,-10.07,-78.15,75.13,7.14,
425,Ambilobe,0,MG,1586659017,82,-13.20,49.05,71.83,4.74,
430,Tessalit,0,ML,1586659018,14,20.20,1.01,77.38,4.32,
471,Ballitoville,0,ZA,1586659029,93,-29.54,31.21,72.00,2.24,
495,Mpophomeni,0,ZA,1586659036,66,-29.57,30.19,72.00,4.97,


In [10]:
hotel_df.count()

City           7
Cloudiness     7
Country        7
Date           7
Humidity       7
Latitude       7
Longitude      7
Temperature    7
Wind Speed     7
Hotel Name     7
dtype: int64

In [11]:
# geocoordinates & searching hotels
# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():
    target_coordinates = f"{row['Latitude']}, {row['Longitude']}"
    target_radius = 5000
    target_type = "lodging"
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    country = row['Country']

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving results for Index {index}: {city}, {country}")
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # print the response url, avoid doing for public github repos in order to avoid exposing key
    # print(response.url)

    # convert response to json
    hotel_data = response.json()
    # pprint(hotel_data)
    
    # extract results
    results = hotel_data['results']
          
    try:
        hotel_name = results[0]['name']
        print(f"Closest hotel to {city} at {latitude}, {longitude} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
    except (KeyError, IndexError):
        print("Missing field........skipping.........")

    print("------------------------------------------------------------")
 

Retrieving results for Index 9: Barra de Tecoanapa, MX
Closest hotel to Barra de Tecoanapa at 16.52, -98.75 is Hotel Manglares.
------------------------------------------------------------
Retrieving results for Index 57: Port Macquarie, AU
Closest hotel to Port Macquarie at -31.43, 152.92 is Rydges Port Macquarie.
------------------------------------------------------------
Retrieving results for Index 82: Huarmey, PE
Closest hotel to Huarmey at -10.07, -78.15 is Hotel Miramar.
------------------------------------------------------------
Retrieving results for Index 425: Ambilobe, MG
Closest hotel to Ambilobe at -13.2, 49.05 is Hôtel & Restaurant DIANA Ambilobe.
------------------------------------------------------------
Retrieving results for Index 430: Tessalit, ML
Missing field........skipping.........
------------------------------------------------------------
Retrieving results for Index 471: Ballitoville, ZA
Closest hotel to Ballitoville at -29.54, 31.21 is Hotel Fairmont Zimb

In [12]:
hotel_df.head()


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed,Hotel Name
9,Barra de Tecoanapa,0,MX,1586658896,82,16.52,-98.75,77.86,4.29,Hotel Manglares
57,Port Macquarie,0,AU,1586658909,37,-31.43,152.92,73.00,6.93,Rydges Port Macquarie
82,Huarmey,0,PE,1586658748,72,-10.07,-78.15,75.13,7.14,Hotel Miramar
425,Ambilobe,0,MG,1586659017,82,-13.20,49.05,71.83,4.74,Hôtel & Restaurant DIANA Ambilobe
430,Tessalit,0,ML,1586659018,14,20.20,1.01,77.38,4.32,


In [13]:
hotel_df.count()

City           7
Cloudiness     7
Country        7
Date           7
Humidity       7
Latitude       7
Longitude      7
Temperature    7
Wind Speed     7
Hotel Name     7
dtype: int64

In [14]:
# drow any rows with null values
# put 'NaN' as hotel name, so that we can drop it later
#hotel_df.loc[index, 'Hotel Name'] = np.nan
# hotel_df = hotel_df[hotel_df.Hotel Name != ""]

#OR 
# drow any rows with null values
filter = hotel_df["Hotel Name"] != ""
hotel_df = hotel_df[filter]
hotel_df


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed,Hotel Name
9,Barra de Tecoanapa,0,MX,1586658896,82,16.52,-98.75,77.86,4.29,Hotel Manglares
57,Port Macquarie,0,AU,1586658909,37,-31.43,152.92,73.00,6.93,Rydges Port Macquarie
82,Huarmey,0,PE,1586658748,72,-10.07,-78.15,75.13,7.14,Hotel Miramar
425,Ambilobe,0,MG,1586659017,82,-13.20,49.05,71.83,4.74,Hôtel & Restaurant DIANA Ambilobe
471,Ballitoville,0,ZA,1586659029,93,-29.54,31.21,72.00,2.24,Hotel Fairmont Zimbali Resort
495,Mpophomeni,0,ZA,1586659036,66,-29.57,30.19,72.00,4.97,Sakkie and friends


In [15]:
hotel_df.count()

City           6
Cloudiness     6
Country        6
Date           6
Humidity       6
Latitude       6
Longitude      6
Temperature    6
Wind Speed     6
Hotel Name     6
dtype: int64

In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map

# fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

# heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("Hotel_Heatmap.png")

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 648x360 with 0 Axes>